<a href="https://colab.research.google.com/github/sanAkel/ssh_tc/blob/main/get_track.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To download track (trajectory) data

## Set up environment.

## Track data comes from [tropycal](https://tropycal.github.io/tropycal/install.html).

In [ ]:
!pip install tropycal
!pip install cartopy

In [ ]:
from tropycal import tracks, rain
import xarray as xr

In [ ]:
# Basin and year
myBasin = 'north_atlantic'
year = 2024
cat_threshold = 4

In [ ]:
# Initialize
# https://tropycal.github.io/tropycal/api/generated/tropycal.tracks.TrackDataset.html#tropycal.tracks.TrackDataset

basin = tracks.TrackDataset(basin=myBasin, source='hurdat', include_btk=True) # include most recent season
season = basin.get_season(year)
print(season)

In [ ]:
season.plot(map_prop={'figsize':(10,6),'linewidth':0.5, 'state_alpha':0.5})
plt.savefig('hurdat2_{}_{}.png'.format(myBasin, year))

In [ ]:
# save data
season_data=season.to_dataframe()
season_data.to_csv('hurdat2_{}_{}.csv'.format(myBasin, year))

### Focus on _major_ (category >=4) storms only

In [ ]:
major_hurr_names = season_data['name'][season_data['category'] >=cat_threshold]
major_hurr_ids = season_data['id'][season_data['category'] >=cat_threshold]

print("\n\nStorms that had a category >= {} are following:\n".format(cat_threshold))
for hurr in major_hurr_names:
    print(hurr)

## Plot those chosen ones

In [ ]:
myTitle = 'North Atlantic basin category >= {}'.format(cat_threshold)

basin.plot_storms(major_hurr_ids,
                  domain=myBasin,
                  title=myTitle,
                  plot_all_dots=True,
                  map_prop={'figsize':(10,6), 'land_color':'w', 'state_alpha':0},
                  prop={'dots':False,'linecolor':'category','linewidth':1,'plot_names':True, 'ms':'6', 'dpi':'120'})

plt.savefig('hurdat2_{}_{}_major.png'.format(myBasin, year))

In [ ]:
# save storm data
for idx, hurr_id in enumerate(major_hurr_ids):
    print("Saving:\t{}, [{}]".format(major_hurr_names.iloc[idx], hurr_id))
    fName = str(year) + "_" + myBasin + "_" + major_hurr_names.iloc[idx] + '.nc'
    basin.get_storm(hurr_id).to_xarray().to_netcdf(fName)

## Move saved files for further analysis

In [ ]:
!ls -al *.nc

In [ ]:
# Mount drive - to save files once done
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
loc_to_save = '/content/drive/MyDrive/datasets/hurr/{}'.format(year)

!mkdir -p $loc_to_save

!cp -r /content/*.csv $loc_to_save

!cp -r /content/*.nc $loc_to_save

!cp -r /content/*.png $loc_to_save

# clean up!
!rm -f /content/*.nc
!rm -f /content/*.csv
!rm -f /content/*.png